# 言語処理100本ノック_第3章: 正規表現
## https://nlp100.github.io/ja/ch03.html

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iamtatsuki05/NLP_100/blob/fix_all_merge/NLP_100_3.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# スペースの修正、変数名修正

# JSONデータの読み込みPermalink
## Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
!wget https://nlp100.github.io/data/jawiki-country.json.gz

In [ ]:
!gunzip ./jawiki-country.json.gz

In [ ]:
!head -1 ./jawiki-country.json

In [ ]:
import json

file_path = 'jawiki-country.json'
with open(file_path, mode='r')as f:
    for line in f:
        line = json.loads(line)
        if line['title'] == 'イギリス':
            text = line['text']
            break

print(text)

In [ ]:
# 参考 https://www.sejuku.net/blog/78985

# カテゴリ名を含む行を抽出
## 記事中でカテゴリ名を宣言している行を抽出せよ．

In [ ]:
import re

pattern = r'^(.*\[\[Category:.*\]\].*)$'
result = '\n'.join(re.findall(pattern, text, re.MULTILINE))
print(result)

In [ ]:
# 参考 https://docs.python.org/ja/3/library/re.html
# https://www.youtube.com/watch?v=Nbdvm36Cau8&t=571s

# カテゴリ名の抽出
## 記事のカテゴリ名を（行単位ではなく名前で）抽出せよ

In [ ]:
pattern = r'^.*\[\[Category:(.*?)(?:\|.*)?\]\].*$'
result = '\n'.join(re.findall(pattern, text, re.MULTILINE))
print(result)

# セクション構造
## 記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [ ]:
pattern = r'^(\={2,})\s*(.+?)\s*(\={2,}).*$'
result = '\n'.join(i[1] + ':' + str(len(i[0]) - 1) for i in re.findall(pattern, text, re.MULTILINE))
print(result)

# ファイル参照の抽出 
## 記事から参照されているメディアファイルをすべて抜き出せ

In [ ]:
pattern = r'\[\[ファイル:(.+?)\|'
result = '\n'.join(re.findall(pattern, text))
print(result)

#  テンプレートの抽出
## 記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ

In [ ]:
#抽出
pattern = r'^\{\{基礎情報.*?$(.*?)^\}\}'
template = re.findall(pattern, text, re.MULTILINE + re.DOTALL)
print(template)

In [ ]:
#辞書に格納
pattern = r'^\|(.+?)\s*=\s*(.+?)(?:(?=\n\|)|(?=\n$))'
result = dict(re.findall(pattern, template[0], re.MULTILINE + re.DOTALL))
for k, v in result.items():
    print(k + ': ' + v)

# 強調マークアップの除去 
## 25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
def remove_markup(text):
    pattern = r'\'{2,5}'
    text = re.sub(pattern, '', text)

    return text


removed_result = {k : remove_markup(v) for k, v in result.items()}
for k, v in removed_result.items():
    print(k + ': ' + v)

# 内部リンクの除去 
## 26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
def remove_markup(text):
    pattern = r'\'{2,5}'
    text = re.sub(pattern, '', text)
    
    pattern = r'\[\[(?:[^|]*?\|)??([^|]*?)\]\]'
    text = re.sub(pattern, r'\1', text)
    
    return text


removed_result = {k: remove_markup(v) for k, v in result.items()}
for k, v in removed_result.items():
    print(k + ': ' + v)

# MediaWikiマークアップの除去 
## 27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [ ]:
def remove_markup(text):
    pattern = r'\'{2,5}'
    text = re.sub(pattern, '', text)

    pattern = r'\[\[(?:[^|]*?\|)??([^|]*?)\]\]'
    text = re.sub(pattern, r'\1', text)

    pattern = r'https?://[\w!?/\+\-_~=;\.,*&@#$%\(\)\'\[\]]+'
    text = re.sub(pattern, '', text)

    pattern = r'<.+?>' 
    text = re.sub(pattern, '', text)

    pattern = r'\{\{(?:lang|仮リンク)(?:[^|]*?\|)*?([^|]*?)\}\}' 
    text = re.sub(pattern, r'\1', text)

    return text


removed_result = {k: remove_markup(v) for k, v in result.items()}
for k, v in removed_result.items():
    print(k + ': ' + v)

# 国旗画像のURLを取得する
## テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [ ]:
import requests

def get_url(text):
    url_file = text['国旗画像'].replace(' ', '_')
    url = 'https://commons.wikimedia.org/w/api.php?action=query&titles=File:' + url_file + '&prop=imageinfo&iiprop=url&format=json'
    data = requests.get(url)
    return re.search(r'"url":"(.+?)"', data.text).group(1)


print(get_url(result))

In [ ]:
# 参考https://www.mediawiki.org/wiki/API:Main_page/ja